In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# The below line is used to hide output of this cell
# %%capture
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate optimum
!pip install -q ipython-autotime
!git lfs install
!pip install pydub
!sudo apt install ffmpeg
!pip install gradio
!pip install pyannote.audio
!pip install gradio
!pip install evaluate
!pip install jiwer

%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 11.3 MB/s eta 0:00:00:00:0100:01
   ━━

In [3]:
# !GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/spaces/hf-audio/whisper-large-v3

time: 302 µs (started: 2025-02-09 08:20:40 +00:00)


In [4]:
import torch
import gradio as gr
from transformers import pipeline

time: 23.9 s (started: 2025-02-09 08:20:40 +00:00)


In [5]:
# # TODO: Init global variables for GG Colab
# import os
# from google.colab import userdata

# FILE_LIMIT_MB = 5000                # Limit to 5Gb video file
# YT_LENGTH_LIMIT_S = 10800           # Limit to 3 hours video length

# HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time: 229 µs (started: 2025-02-09 08:21:04 +00:00)


In [6]:
# TODO: Init global variables for Kaggle
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

FILE_LIMIT_MB = 5000                # Limit to 5Gb video file
YT_LENGTH_LIMIT_S = 10800           # Limit to 3 hours video length

HUGGINGFACE_TOKEN = user_secrets.get_secret("HUGGINGFACE_TOKEN")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

time: 251 ms (started: 2025-02-09 08:21:04 +00:00)


# Pipeline

In [7]:
# TODO: Configure a pipeline of ASR
pipe = pipeline("automatic-speech-recognition",
                "openai/whisper-large-v3",
                torch_dtype=torch.float16,
                device=device)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda


time: 1min 19s (started: 2025-02-09 08:21:04 +00:00)


In [8]:
# TODO: Configure Speaker-Diarization pipeline
from pyannote.audio import Pipeline

diarizer = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=HUGGINGFACE_TOKEN
)

diarizer.to(device)

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

time: 7.19 s (started: 2025-02-09 08:22:23 +00:00)


In [9]:
# TODO: Collect metadata for audio input
from pydub import AudioSegment
from pydub.utils import mediainfo

def get_metadata(file_path):
    metadata = {}
    info = mediainfo(file_path)
    duration = get_duration_pydub(file_path)

    interest_info = ["sample_rate", "format_name", "channels", "filename", "size"]
    interest_values = [info[k] for k in interest_info]

    metadata['duration'] = duration
    metadata.update(dict(zip(interest_info, interest_values)))

    return metadata

def get_duration_pydub(file_path):
   audio_file = AudioSegment.from_file(file_path)
   duration = audio_file.duration_seconds
   return duration


time: 683 µs (started: 2025-02-09 08:22:30 +00:00)


In [ ]:
# TODO: Transcribe pipeline for Youtube URL
# import yt_dlp as youtube_dl
# import tempfile
# from transformers.pipelines.audio_utils import ffmpeg_read

# def download_yt_audio(yt_url, filename):
#     info_loader = youtube_dl.YoutubeDL()

#     try:
#         info = info_loader.extract_info(yt_url, download=False)
#     except youtube_dl.utils.DownloadError as err:
#         raise gr.Error(str(err))

#     ydl_opts = {"outtmpl": filename, "format": "worstvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best"}
#     with youtube_dl.YoutubeDL(ydl_opts) as ydl:
#         try:
#             ydl.download([yt_url])
#         except youtube_dl.utils.ExtractorError as err:
#             raise gr.Error(str(err))

# def yt_transcribe(yt_url):

#     with tempfile.TemporaryDirectory() as tmpdirname:
#         filepath = os.path.join(tmpdirname, "video.mp4")
#         download_yt_audio(yt_url, filepath)
#         with open(filepath, "rb") as f:
#             inputs = f.read()

#     inputs = ffmpeg_read(inputs, pipe.feature_extractor.sampling_rate)
#     inputs = {"array": inputs, "sampling_rate": pipe.feature_extractor.sampling_rate}

#     output = pipe(
#         inputs,
#         generate_kwargs={
#             "task": "transcribe"
#         },
#         chunk_length_s=24,
#         batch_size=32,
#         return_timestamps=True
#     )

#     return output["text"], output['chunks']

time: 726 µs (started: 2025-02-09 08:23:03 +00:00)


In [12]:
# TODO: Utils function
from pydub import AudioSegment
import os

def check_file_extension(file_path, allowed_extensions=None):
    """
    Check the extension of a file.
    Args:
        file_path: str, path or name of the file to check
        allowed_extensions: list, optional, list of allowed extensions (e.g., ['.jpg', '.png'])

    Return:
            str or bool, the file extension if no allowed_extensions is provided,
             or True/False indicating whether the file has a valid extension
    """
    # Get the file extension
    _, extension = os.path.splitext(file_path)
    extension = extension.lower()  # Normalize to lowercase for consistent comparison

    # If allowed extensions are provided, validate
    if allowed_extensions:
        allowed_extensions = [ext.lower() for ext in allowed_extensions]
        return extension in allowed_extensions

    return extension

def extract_audio_to_wav(input_path, output_path):
    """
    Extract audio from a video file and save it as a .wav file.

    Parameters:
        input_path (str): Path to the input video file (e.g., .mp4, .m4a).
        output_path (str): Path to save the extracted audio as a .wav file.
    """
    try:
        audio = AudioSegment.from_file(input_path)

        audio.export(output_path, format="wav")
        print(f"Audio successfully extracted to: {output_path}")
    except Exception as e:
        print(f"Error extracting audio: {e}")

time: 18.2 ms (started: 2025-02-09 08:23:03 +00:00)


In [13]:
# TODO: Diarization

# def diarize_audio_from_yt(yt_url, num_speakers=0, min_speakers=0, max_speakers=0):
#     try:
#         params = {}
#         if num_speakers > 0:
#             params["num_speakers"] = num_speakers
#         if min_speakers > 0:
#             params["min_speakers"] = min_speakers
#         if max_speakers > 0:
#             params["max_speakers"] = max_speakers

#         with tempfile.TemporaryDirectory() as tmpdirname:
#             filepath = os.path.join(tmpdirname, "video.mp4")
#             download_yt_audio(yt_url, filepath)
#             audio_path = os.path.join(tmpdirname, "audio.wav")
#             extract_audio_to_wav(filepath, audio_path)

#             diarization = diarizer(audio_path, **params)

#     except Exception as e:
#         return f"Error processing audio: {e}"

#     return diarization

def diarize_audio_from_file(filename, num_speakers=0, min_speakers=0, max_speakers=0):

    # Parse number of speakers
    try:
        params = {}
        if num_speakers > 0:
            params["num_speakers"] = num_speakers
        if min_speakers > 0:
            params["min_speakers"] = min_speakers
        if max_speakers > 0:
            params["max_speakers"] = max_speakers

    except Exception as e:
        return f"Error processing audio: {e}"

    # Check if it is a video or audio
    is_video = check_file_extension(filename, [".mp4"])
    is_audio = check_file_extension(filename, [".mp3", ".wav"])

    if is_video:
        with tempfile.TemporaryDirectory() as tmpdirname:
            audio_path = os.path.join(tmpdirname, "audio.wav")
            extract_audio_to_wav(filename, audio_path)

            diarization = diarizer(audio_path, **params)
    elif is_audio:
        diarization = diarizer(filename, **params)

    return diarization

time: 11.7 ms (started: 2025-02-09 08:23:03 +00:00)


In [14]:
# TODO: Combine outputs from Caption Recognition and Diarization

def parse_diarization(diarization):
    segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segment = {}
        segment['Speaker'] = speaker
        segment['start'] = round(turn.start, 1) # Rounded start time to .1f
        segment['end'] = round(turn.end, 1)

        segments.append(segment)

    return segments

def merge_speaker_segments(segments):
    merged_segments = [segments[0]]

    for current in segments[1:]:
        last = merged_segments[-1]

        if current['Speaker'] == last["Speaker"]:
            new_start = last["start"]
            new_end = current['end']

            new_segment = {
                "Speaker": current["Speaker"],
                "start": new_start,
                "end": new_end
            }

            merged_segments.pop(-1)
            merged_segments.append(new_segment)

        elif current['Speaker'] != last["Speaker"]:
            merged_segments.append(current)

    return merged_segments

time: 13.6 ms (started: 2025-02-09 08:23:03 +00:00)


In [15]:
filename = "/kaggle/input/toy-audio/speaking1_2person.mp3"
diarization = diarize_audio_from_file(filename)

/usr/local/lib/python3.10/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)
/usr/local/lib/python3.10/dist-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribu

time: 42 s (started: 2025-02-09 08:23:03 +00:00)


In [16]:
segments = parse_diarization(diarization)
result = merge_speaker_segments(segments)
result[:5]

[{'Speaker': 'SPEAKER_00', 'start': 0.2, 'end': 9.0},
 {'Speaker': 'SPEAKER_01', 'start': 9.2, 'end': 26.2},
 {'Speaker': 'SPEAKER_00', 'start': 26.6, 'end': 29.3},
 {'Speaker': 'SPEAKER_01', 'start': 29.7, 'end': 46.3},
 {'Speaker': 'SPEAKER_00', 'start': 46.9, 'end': 49.9}]

time: 3.1 ms (started: 2025-02-09 08:23:45 +00:00)


In [17]:
# TODO: Perform Caption Recognition from file uploaded
def transcribe_speech(filepath):
    output = pipe(
        filepath,
        generate_kwargs={
            "task": "transcribe"
        },
        chunk_length_s=24,
        batch_size=32,
        return_timestamps=True
    )
    return output["text"], output['chunks']

time: 12.2 ms (started: 2025-02-09 08:23:45 +00:00)


In [18]:
filename = "/kaggle/input/toy-audio/speaking1_2person.mp3"
text, chunks = transcribe_speech(filename)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


time: 19.5 s (started: 2025-02-09 08:31:47 +00:00)


In [19]:
chunks[:8]

[{'timestamp': (0.0, 4.48),
  'text': ' Welcome to the IELTS Speaking Test. In this first part, I would like to get to know you better.'},
 {'timestamp': (4.48, 9.04),
  'text': " So first of all, let's talk about language. What languages can you speak?"},
 {'timestamp': (9.04, 13.28),
  'text': " Actually, I'm a bilingual. So firstly, my mother tongue Vietnamese,"},
 {'timestamp': (13.28, 18.88),
  'text': ' and the second one is English. And I suppose that on the ground that English'},
 {'timestamp': (18.88, 22.36),
  'text': ' is an international language so almost all'},
 {'timestamp': (22.36, 26.76),
  'text': " the young people could communicate in English and I'm no exception."},
 {'timestamp': (26.76, 29.88),
  'text': ' What languages would you like to learn in the future?'},
 {'timestamp': (29.88, 34.24),
  'text': ' On the top of my head when talking about this topic maybe Chinese.'}]

time: 2.67 ms (started: 2025-02-09 08:32:07 +00:00)


In [20]:
result[:5]

[{'Speaker': 'SPEAKER_00', 'start': 0.2, 'end': 9.0},
 {'Speaker': 'SPEAKER_01', 'start': 9.2, 'end': 26.2},
 {'Speaker': 'SPEAKER_00', 'start': 26.6, 'end': 29.3},
 {'Speaker': 'SPEAKER_01', 'start': 29.7, 'end': 46.3},
 {'Speaker': 'SPEAKER_00', 'start': 46.9, 'end': 49.9}]

time: 12.1 ms (started: 2025-02-09 08:32:07 +00:00)


In [21]:
def check_chunk_and_segment_match(segment, chunk):
    percent_overlap = calculate_percent_overlap(segment, chunk)
    return is_match(percent_overlap)

def is_match(percent_overlap):
    return True if percent_overlap >= 0.80 else False

def calculate_percent_overlap(seg1, seg2):
    """Calculate the overlap between two intervals in percentage."""
    
    start1, end1 = seg1
    start2, end2 = seg2
    if end2 is None:
        end2 = float('inf') # Handle the case: End of conversation
        
    overlap_start = max(start1, start2)
    overlap_end = min(end1, end2)
    overlap_length = max(0, overlap_end - overlap_start)
    
    # Percent overlap relative to chunk 
    percent_overlap = overlap_length / (end2 - start2) 

    return percent_overlap

def extract_speaker_sentences(segments, chunks):
    """Match segments to chunks based on overlap."""
    results = []
    conversation = []
    
    for segment in segments:
        speaker = segment['Speaker']
        speaker_interval = (segment['start'], segment['end'])
        
        for chunk in chunks:
            text = chunk['text']
            chunk_timestamp = chunk['timestamp']

            if check_chunk_and_segment_match(speaker_interval, chunk_timestamp):
                conversation.append({
                    'Speaker': speaker,
                    'text': text,
                    'segment': speaker_interval,
                    'timestamp': chunk_timestamp
                })
    return conversation

time: 13.4 ms (started: 2025-02-09 08:32:07 +00:00)


In [22]:
def compress_conversation(speaker_parts):
    compressed = []
    for part in speaker_parts:
        if compressed and compressed[-1]["Speaker"] == part["Speaker"]:
            # Merge text for the same speaker
            compressed[-1]["text"] += "" + part["text"]
        else:
            # Add a new entry with only Speaker and text fields
            compressed.append({"Speaker": part["Speaker"], "text": part["text"]})
    return compressed
    
def display_script(compressed_parts):
    formatted_output = ""
    for part in compressed_parts:
        formatted_output += f"{part['Speaker']}:\n"
        formatted_output += f"    - {part['text']}\n\n"  # Add indentation and blank line
    return formatted_output.strip()

time: 18.8 ms (started: 2025-02-09 08:32:07 +00:00)


In [23]:
conversation = extract_speaker_sentences(result, chunks)
conversation[12:16]

[{'Speaker': 'SPEAKER_01',
  'text': ' My university has been training various languages. Besides',
  'segment': (50.1, 73.9),
  'timestamp': (50.26, 56.0)},
 {'Speaker': 'SPEAKER_01',
  'text': ' delivering the lecture, the professors also promote interactive learning and continuous assessment.',
  'segment': (50.1, 73.9),
  'timestamp': (56.0, 63.12)},
 {'Speaker': 'SPEAKER_01',
  'text': ' The students also can acquire the knowledge about pronunciation, grammar, and also have',
  'segment': (50.1, 73.9),
  'timestamp': (64.08, 71.6)},
 {'Speaker': 'SPEAKER_01',
  'text': " Actually repairing something requires a tough nut to crack for me especially for a girl so I can't",
  'segment': (78.6, 90.0),
  'timestamp': (78.96, 88.0)}]

time: 22 ms (started: 2025-02-09 08:32:07 +00:00)


In [24]:
compressed_parts = compress_conversation(conversation)
final = display_script(compressed_parts)
compressed_parts[5:7]

[{'Speaker': 'SPEAKER_01',
  'text': " My university has been training various languages. Besides delivering the lecture, the professors also promote interactive learning and continuous assessment. The students also can acquire the knowledge about pronunciation, grammar, and also have Actually repairing something requires a tough nut to crack for me especially for a girl so I can't So when I was a little kid like all the curious, I always saw the opportunity to explore the whole world. So when things broke, my parents were always willing to help me to fix, to teach me to repair things."},
 {'Speaker': 'SPEAKER_00',
  'text': ' What do you do when a thing is broken and cannot be fixed?'}]

time: 14.3 ms (started: 2025-02-09 08:32:07 +00:00)


In [25]:
# TODO: Post-processing
def extract_diarized_text(text: list[dict]):
    diarized_text = ""
    
    for segment in text:
        diarized_text += segment['text']
    
    return diarized_text

def get_unique_speakers_name(text: list[dict], return_amount=False):
    speakers = []
    for chunk in text:
        speakers.append(chunk['Speaker'])
    if return_amount:
        n = len(set(speakers))
        return sorted(set(speakers)), n
    else:
        return sorted(set(speakers))

time: 9.46 ms (started: 2025-02-09 08:32:07 +00:00)


In [26]:
speakers, nspeakers = get_unique_speakers_name(compressed_parts, return_amount=True)

time: 10.2 ms (started: 2025-02-09 08:32:07 +00:00)


# Post-processing

In [27]:
def detect_missing_parts(original: str, comparison: str):
    """
    Detect missing words or phrases between two paragraphs.

    Parameters:
        original (str): The reference paragraph.
        comparison (str): The paragraph to compare with the reference.

    Returns:
        missing_phrases (list): List of missing words or phrases.
        missing_locations (list): List of start indices for each missing part in the original paragraph.
    """
    # Split the paragraphs into words
    original_words = original.split()
    comparison_words = comparison.split()

    missing_phrases = []
    missing_locations = []

    # Pointer for comparison_words
    comp_index = 0
    missing_buffer = []  # To track consecutive missing words

    for i, word in enumerate(original_words):
        if comp_index < len(comparison_words) and word == comparison_words[comp_index]:
            # Word matches, flush the buffer if any missing words were tracked
            if missing_buffer:
                missing_phrases.append(" ".join(missing_buffer))
                missing_locations.append(i - len(missing_buffer))
                missing_buffer = []  # Clear buffer
            comp_index += 1  # Move to next word in comparison_words
        else:
            # Word is missing in comparison_words
            missing_buffer.append(word)

    # Flush the buffer for any trailing missing words
    if missing_buffer:
        missing_phrases.append(" ".join(missing_buffer))
        missing_locations.append(len(original_words) - len(missing_buffer))

    return missing_phrases, missing_locations

time: 3.5 ms (started: 2025-02-09 08:32:07 +00:00)


In [28]:
text[:1500]

" Welcome to the IELTS Speaking Test. In this first part, I would like to get to know you better. So first of all, let's talk about language. What languages can you speak? Actually, I'm a bilingual. So firstly, my mother tongue Vietnamese, and the second one is English. And I suppose that on the ground that English is an international language so almost all the young people could communicate in English and I'm no exception. What languages would you like to learn in the future? On the top of my head when talking about this topic maybe Chinese. Because I see that the number of people on around the world who could communicate and speak in Chinese is increasing. So maybe like learning Chinese or being left behind. How are languages taught and learned in your school? My university has been training various languages. Besides delivering the lecture, the professors also promote interactive learning and continuous assessment. The students also can acquire the knowledge about pronunciation, gra

time: 16.1 ms (started: 2025-02-09 08:32:07 +00:00)


In [29]:
diarized_text = extract_diarized_text(compressed_parts)
diarized_text[:1000]

" Welcome to the IELTS Speaking Test. In this first part, I would like to get to know you better. So first of all, let's talk about language. What languages can you speak? Actually, I'm a bilingual. So firstly, my mother tongue Vietnamese, and the second one is English. And I suppose that on the ground that English is an international language so almost all the young people could communicate in English and I'm no exception. What languages would you like to learn in the future? On the top of my head when talking about this topic maybe Chinese. Because I see that the number of people on around the world who could communicate and speak in Chinese is increasing. So maybe like learning Chinese or being left behind. How are languages taught and learned in your school? My university has been training various languages. Besides delivering the lecture, the professors also promote interactive learning and continuous assessment. The students also can acquire the knowledge about pronunciation, gra

time: 11.8 ms (started: 2025-02-09 08:32:07 +00:00)


In [30]:
missing_phrases, missing_locations = detect_missing_parts(text, diarized_text)
print(missing_phrases)
print(missing_locations[0])

["a chance to explore the new culture. Now let's move on to talk about fixing things. Can you fix things?", 'hardly fix anything. Did anybody teach you to fix things when you were a child?', "Now let's talk about bags. Do you usually carry a bag? Always, absolutely. Like bags is my", 'Is a bag an ideal gift? Well, from my perspective, bags could be a practical and functional gift for other people.', 'I stop you. You can start now.', 'When is the right time for young people to start making serious plans for their future careers?']
170
time: 12.3 ms (started: 2025-02-09 08:32:07 +00:00)


In [31]:
from jiwer import compute_measures

measures = compute_measures(text, diarized_text)
print(measures['substitutions'])
print(measures["deletions"])
print(measures["insertions"])
print(measures["hits"])
print(measures['wer'])

0
97
0
947
0.09291187739463602
time: 32.6 ms (started: 2025-02-09 08:32:07 +00:00)


In [32]:
from evaluate import load

wer = load("wer")
wer_score = wer.compute(predictions=[diarized_text], references=[text])
wer_score_percent = round(wer_score * 100, 3) 
print(f"{wer_score_percent}%")

9.291%
time: 1.6 s (started: 2025-02-09 08:32:07 +00:00)


In [33]:
custom_css = """
#custom_button {
    background-color: orange !important;
    color: white !important;
    font-size: 18px !important;
    padding: 10px 20px !important;
    border-radius: 8px !important;
    border: none !important;
    display: block;
    margin: auto;
    width: 250px;
    text-align: center;
}
"""

time: 295 µs (started: 2025-02-09 08:32:09 +00:00)


# CWD

In [34]:
def update_transcript_names(transcript, new_names:list[str]):
    """
    Replace each default speaker label with the new name provided.
    
    Parameters:
      transcript (str): The original transcript text.
      new_names (list[str]): A list of new names corresponding to the default speakers in order.
    
    Returns:
      The transcript with speaker names updated.
    """
    for i, new_name in enumerate(new_names):
        default_label = f"SPEAKER_{i:02d}"
        transcript = transcript.replace(default_label, new_name)
    return transcript

time: 12.6 ms (started: 2025-02-09 08:32:09 +00:00)


In [35]:
t = update_transcript_names(final, new_names=['Sam', 'Teddy'])

time: 10.1 ms (started: 2025-02-09 08:32:09 +00:00)


In [36]:
#### Functions for Gradio
def update_transcript_names(transcript, new_names:list[str]):
    """
    Replace each default speaker label with the new name provided.
    
    Parameters:
      transcript (str): The original transcript text.
      new_names (list[str]): A list of new names corresponding to the default speakers in order.
    
    Returns:
      The transcript with speaker names updated.
    """
    for i, new_name in enumerate(new_names):
        default_label = f"SPEAKER_{i:02d}"
        transcript = transcript.replace(default_label, new_name)
    return transcript


def display_file(file):
    if file is None:
        return None
    return file.name  # Returns the file path to display in the preview

def transcribe_from_file(filename):
    texts, chunks = transcribe_speech(filename)
    diarization = diarize_audio_from_file(filename)
    segments = parse_diarization(diarization)
    merged_segments = merge_speaker_segments(segments)
    conversation = extract_speaker_sentences(merged_segments, chunks)
    compressed_conv = compress_conversation(conversation)
    script = display_script(compressed_conv)
    
    return script

def update_transcript_names(transcript, new_names):
    """
    Replace each default speaker label with the new name provided.
    
    Parameters:
      transcript (str): The original transcript text.
      new_names (list[str]): A list of new names corresponding to the default speakers in order.
    
    Returns:
      The transcript with speaker names updated.
    """
    for i, new_name in enumerate(new_names):
        default_label = f"SPEAKER_{i:02d}"
        transcript = transcript.replace(default_label, new_name)
    return transcript

def process_export(transcript, *new_names):
    """
    Process the transcript update and write the new transcript to a text file.
    Returns the file path for Gradio to enable file download.
    """
    updated_transcript = update_transcript_names(transcript, new_names)
    file_path = "edited_transcript.txt"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(updated_transcript)
    return file_path

time: 7.62 ms (started: 2025-02-09 08:32:09 +00:00)


In [37]:
speakers, nspeakers = get_unique_speakers_name(compressed_parts, return_amount=True)
speakers

['SPEAKER_00', 'SPEAKER_01']

time: 14 ms (started: 2025-02-09 08:32:09 +00:00)


# Class

In [ ]:
class Transcriber:
    def __init__(self,
                model_name="openai/whisper-large-v3",
                torch_dtype=torch.float16,
                device='cuda'):
        
        # TODO: Configure a pipeline of ASR
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.transcriber = pipeline("automatic-speech-recognition",
                            model_name,
                            torch_dtype=torch_dtype,
                            device=self.device)

    def transcribe_speech(self, filepath, chunk_length_s=24, batch_size=32, return_timestamps=True):
        output = self.transcriber(
                    filepath,
                    generate_kwargs={
                        "task": "transcribe"
                    },
                    chunk_length_s=chunk_length_s,
                    batch_size=batch_size,
                    return_timestamps=return_timestamps
                )
        return output["text"], output['chunks']



In [ ]:
class AutoScriptingPipeline:
    def __init__(self, )

# Gradio

In [38]:
import gradio as gr

# Layout
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as app:
    gr.Markdown(
        """
        <div style="text-align: center; font-size: 24px; font-weight: bold;">
            Interview/Podcast Transcriptor
        </div>
        """
    )
    gr.Markdown("#### Preview")
    with gr.Sidebar():
        gr.Markdown("#### Upload file")
        file_input = gr.File(label="Upload Video/Audio", file_types=[".mp4", ".mp3", ".wav", ".flac"])
        
        num_speaker = gr.Number(label="Number of speakers:", minimum=0, maximum=10, value=0, step=1)
        
        min_speaker = gr.Number(
            label="Minimum number of speakers:", 
            minimum=0, maximum=10, 
            value=0, step=1, 
            info="Set to 0 to auto-detect")

        max_speaker = gr.Number(
            label="Maximum number of speakers:", 
            minimum=0, maximum=10, 
            value=0, step=1, 
            info="Set to 0 to auto-detect")
        
    with gr.Row():
        with gr.Column():
            file_preview = gr.Video(label="Preview", visible=True, width=600)
            file_input.change(display_file, file_input, file_preview)
            process_button = gr.Button("Transcribe", elem_id="custom_button")
            
    with gr.Row():
        with gr.Column(scale=4):
            text_output = gr.Textbox(label="Script")

        speaker_textboxes = []
        with gr.Column(scale=1):
            for speaker in speakers:
                textbox = gr.Textbox(value=speaker, label=f"Rename {speaker}")
                speaker_textboxes.append(textbox)
                
            export_button = gr.Button("Export")
            download_btn = gr.File(label="Download Exported File Here...")
            
    # Connect process function to the button
    process_button.click(transcribe_from_file, inputs=[file_input], outputs=text_output)
    export_button.click(
        process_export, 
        inputs=[text_output] + speaker_textboxes, 
        outputs=download_btn
    )

# Launch the app
app.launch()


* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://80a328e491fd60a435.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


time: 1.55 s (started: 2025-02-09 08:32:09 +00:00)


In [41]:
import gradio as gr

# Simulated function for transcribing speech and detecting speakers
def transcribe_video(video):
    # Simulated output
    transcript = """SPEAKER_00: Hello, welcome to our podcast.\n
SPEAKER_01: Thank you, happy to be here.\n
SPEAKER_02: Let's dive into the topic.\n"""
    
    detected_speakers = ["SPEAKER_00", "SPEAKER_01", "SPEAKER_02"]  # Simulated detected speakers
    return transcript, detected_speakers

# Function to create dynamic input fields for speaker names
def generate_speaker_inputs(detected_speakers):
    inputs = [gr.Textbox(label=f"Rename {spk}", value=spk) for spk in detected_speakers]
    return inputs

# Function to update transcript with new speaker names
def update_transcript(transcript, *speaker_names):
    lines = transcript.split("\n")
    mapping = {f"SPEAKER_{i:02d}": name for i, name in enumerate(speaker_names)}

    updated_lines = []
    for line in lines:
        for old_name, new_name in mapping.items():
            line = line.replace(old_name, new_name)
        updated_lines.append(line)
    
    return "\n".join(updated_lines)

with gr.Blocks() as demo:
    gr.Markdown("# 🎙 Podcast/Interview Auto-Transcription")
    
    # Upload video file
    video_input = gr.Video(label="Upload Video")
    transcribe_button = gr.Button("Transcribe")
    
    # Outputs
    transcript_output = gr.Textbox(label="Transcript", interactive=True)
    speaker_inputs_container = gr.Column(visible=False)  # Container for dynamic speaker name inputs
    update_button = gr.Button("Update Names", visible=False)
    
    # Step 1: Transcribe video and get speakers
    def transcribe_and_display(video):
        transcript, detected_speakers = transcribe_video(video)
        speaker_inputs = generate_speaker_inputs(detected_speakers)
        return transcript, gr.Column(visible=True, children=speaker_inputs), gr.Button(visible=True)

    transcribe_button.click(
        fn=transcribe_and_display,
        inputs=[video_input],
        outputs=[transcript_output, speaker_inputs_container, update_button]
    )
    
    # Step 2: Update transcript with new speaker names
    def rename_speakers(transcript, *names):
        return update_transcript(transcript, *names)

    update_button.click(
        fn=rename_speakers,
        inputs=[transcript_output] + speaker_inputs_container,
        outputs=[transcript_output]
    )

demo.launch()


TypeError: can only concatenate list (not "Column") to list

time: 151 ms (started: 2025-02-09 08:58:50 +00:00)


### Deprecated

In [39]:
# import gradio as gr

# demo = gr.Blocks()


# file_transcribe = gr.Interface(
#     fn=transcribe_from_file,
#     inputs=gr.File(label="Upload Your File", file_count='single', file_types=['.mp4', '.mp3', '.wav', '.flac']),
#     outputs=gr.components.Textbox(),
#     fill_width=True,
#     title="Interview/Podcast Transcriptor",
#     flagging_mode="never"
# )

# # yt_transcribe = gr.Interface(
# #     fn=yt_transcribe,
# #     inputs=[
# #         gr.Textbox(lines=1, placeholder="Paste the URL to a YouTube video here", label="YouTube URL")
# #     ],
# #     outputs=["text"],
# #     title="Whisper Large V3: Transcribe YouTube",
# #     flagging_mode="never",
# # )

time: 195 µs (started: 2025-02-09 08:32:10 +00:00)
